## Aggregation

- Convert scores to pass fail
-

In [2]:
%sql
DROP TABLE IF EXISTS answers_by_attempt;
CREATE TABLE answers_by_attempt AS (
    SELECT a.learner_assignment_attempt_id,
           count(a.assessment_item_response_id)   number_of_distinct_instance_items_answered
    FROM (
        SELECT DISTINCT learner_assignment_attempt_id, learner_assigned_item_attempt_id, assessment_item_response_id
        FROM clean_data
        WHERE learner_attempt_status = 'fully scored'
    ) a
    GROUP BY a.learner_assignment_attempt_id
)

In [3]:
spark.sql("SELECT * FROM answers_by_attempt").printSchema()


root
-- learner_assignment_attempt_id: integer (nullable = true)
-- number_of_distinct_instance_items_answered: long (nullable = true)

In [4]:
%sql
DROP TABLE IF EXISTS scores;
CREATE TABLE scores AS (
   SELECT DISTINCT (cl.learner_assignment_attempt_id)  AS attempt_id,
                         cl.assessment_id,
                         cl.learner_id,
                         cl.section_id,
                         cl.org_id,
                         cl.final_score_unweighted,
                         cl.points_possible_unweighted,
                         cl.was_fully_scored_datetime,
                         cl.number_of_distinct_instance_items, -- number of questions
                         aba.number_of_distinct_instance_items_answered
         FROM clean_data cl
         LEFT JOIN answers_by_attempt aba ON cl.learner_assignment_attempt_id = aba.learner_assignment_attempt_id
         WHERE learner_attempt_status = 'fully scored'
);

In [5]:
spark.sql("SELECT * FROM scores").printSchema()


root
-- attempt_id: integer (nullable = true)
-- assessment_id: integer (nullable = true)
-- learner_id: integer (nullable = true)
-- section_id: integer (nullable = true)
-- org_id: integer (nullable = true)
-- final_score_unweighted: integer (nullable = true)
-- points_possible_unweighted: double (nullable = true)
-- was_fully_scored_datetime: timestamp (nullable = true)
-- number_of_distinct_instance_items: integer (nullable = true)
-- number_of_distinct_instance_items_answered: long (nullable = true)

In [6]:
%sql
DROP TABLE IF EXISTS learners;
CREATE TABLE learners AS (
         SELECT learner_id,
                section_id,
                org_id,
                MIN( DATE(was_fully_scored_datetime) ) AS min_was_fully_scored_date,
                MAX( DATE(was_fully_scored_datetime) ) AS max_was_fully_scored_date,
                DATEDIFF( MAX(DATE(was_fully_scored_datetime)), MIN(DATE(was_fully_scored_datetime)) ) AS days
         FROM clean_data
         GROUP BY learner_id, section_id, org_id
);

In [7]:
%sql
DROP TABLE IF EXISTS sections;
CREATE TABLE sections AS (
         SELECT section_id,
                MIN(DATE(was_fully_scored_datetime)) AS min_was_fully_scored_date,
                MAX(DATE(was_fully_scored_datetime)) AS max_was_fully_scored_date,
                DATEDIFF( MAX(DATE(was_fully_scored_datetime)), MIN(DATE(was_fully_scored_datetime)) ) AS days
         FROM clean_data
         GROUP BY section_id
     );

In [8]:
%sql
DROP TABLE IF EXISTS assessments;
CREATE TABLE assessments AS (
         SELECT assessment_id,
                MIN(DATE(was_fully_scored_datetime)) AS min_was_fully_scored_date,
                MAX(DATE(was_fully_scored_datetime)) AS max_was_fully_scored_date,
                DATEDIFF( MAX(DATE(was_fully_scored_datetime)), MIN(DATE(was_fully_scored_datetime)) ) AS days
         FROM clean_data
         GROUP BY assessment_id
         )

In [9]:
%sql
DROP TABLE IF EXISTS orgs;
CREATE TABLE orgs AS (
         SELECT org_id,
                MIN(DATE(was_fully_scored_datetime)) AS min_was_fully_scored_date,
                MAX(DATE(was_fully_scored_datetime)) AS max_was_fully_scored_date,
                DATEDIFF( MAX(DATE(was_fully_scored_datetime)), MIN(DATE(was_fully_scored_datetime)) ) AS days
         FROM clean_data
         GROUP BY org_id
     )

In [10]:
%sql
DROP TABLE IF EXISTS score_by_learner;
CREATE TABLE score_by_learner AS ( -- How learners performed on all assessments attempts
         SELECT l.learner_id, --1126
                ROUND(AVG(s.final_score_unweighted)) AS learner_final_score_unweighted,
                ROUND(AVG(s.points_possible_unweighted)) AS learner_points_possible_unweighted,
                ROUND(AVG(s.number_of_distinct_instance_items_answered)) AS learner_number_of_distinct_instance_items_answered,
                ROUND(AVG(s.number_of_distinct_instance_items)) AS learner_number_of_distinct_instance_items,
                COUNT(*) AS learner_num_attempts,
                ROUND(AVG(l.days)) AS learner_days -- The number of days the learner took assessments

         FROM learners l,
              scores s
         WHERE l.learner_id = s.learner_id
         GROUP BY l.learner_id
     )

In [11]:
%sql
DROP TABLE IF EXISTS score_by_assessment;
CREATE TABLE score_by_assessment  AS ( -- How all learners performed on attempts of an assessment
         SELECT a.assessment_id, -- 329
                ROUND(AVG(s.final_score_unweighted)) AS assessment_final_score_unweighted,
                ROUND(AVG(s.points_possible_unweighted)) AS assessment_points_possible_unweighted,
                ROUND(AVG(s.number_of_distinct_instance_items_answered)) AS assessment_number_of_distinct_instance_items_answered,
                ROUND(AVG(s.number_of_distinct_instance_items)) AS assessment_number_of_distinct_instance_items,
                COUNT(*) AS assessment_num_attempts
         FROM assessments a,
              scores s
         WHERE a.assessment_id = s.assessment_id
         GROUP BY a.assessment_id
     )

In [12]:
%sql
DROP TABLE IF EXISTS score_by_section;
CREATE TABLE score_by_section  AS ( -- How all learners performed on attempts of an assessment by section
         SELECT a.section_id, --490
                s.assessment_id,
                ROUND(AVG(s.final_score_unweighted)) AS section_final_score_unweighted,
                ROUND(AVG(s.points_possible_unweighted)) AS section_points_possible_unweighted,
                ROUND(AVG(s.number_of_distinct_instance_items_answered)) AS section_number_of_distinct_instance_items_answered,
                ROUND(AVG(s.number_of_distinct_instance_items)) AS section_number_of_distinct_instance_items,
                COUNT(*) AS section_num_attempts,
                ROUND(AVG(a.days)) AS section_days
         FROM sections a,
              scores s
         WHERE a.section_id = s.section_id
         GROUP BY a.section_id, s.assessment_id
     )

In [13]:
%sql
DROP TABLE IF EXISTS score_by_org;
CREATE TABLE score_by_org  AS (
         SELECT a.org_id, --329
                s.assessment_id,
                ROUND(AVG(s.final_score_unweighted)) AS organization_final_score_unweighted,
                ROUND(AVG(s.points_possible_unweighted)) AS organization_points_possible_unweighted,
                ROUND(AVG(s.number_of_distinct_instance_items_answered)) AS organization_number_of_distinct_instance_items_answered,
                ROUND(AVG(s.number_of_distinct_instance_items)) AS organization_number_of_distinct_instance_items,
                COUNT(*) AS organization_num_attempts,
                ROUND(AVG(a.days)) AS organization_days
         FROM orgs a,
              scores s
         WHERE a.org_id = s.org_id
         GROUP BY a.org_id, s.assessment_id
     )

In [14]:
%sql
SELECT attempt_id,
       assessment_id,
       learner_id,
       section_id,
       org_id,
       was_fully_scored_datetime,
       final_score_unweighted,
       points_possible_unweighted,
       number_of_distinct_instance_items,
       number_of_distinct_instance_items_answered
FROM scores s LIMIT 10;

attempt_id,assessment_id,learner_id,section_id,org_id,was_fully_scored_datetime,final_score_unweighted,points_possible_unweighted,number_of_distinct_instance_items,number_of_distinct_instance_items_answered
1057378,705605,1295687,498959,349115,2020-02-25T15:43:35.000+0000,5,11.0,11,11
1038094,724435,1248322,498947,360541,2019-09-06T01:03:38.407+0000,12,12.0,1,1
977942,747722,1248377,499028,360421,2019-10-25T16:54:19.313+0000,3,20.0,20,20
950947,678303,1248284,498928,349115,2019-12-04T19:25:34.000+0000,9,10.0,10,10
994367,649459,1248306,498953,360541,2019-08-30T19:18:51.243+0000,8,20.0,10,10
1086424,690485,1248733,498943,349115,2020-01-28T13:58:29.000+0000,20,20.0,20,20
1025806,752434,1372797,522071,360421,2020-04-09T16:43:02.000+0000,6,12.0,12,12
1023479,693282,1248741,498978,360541,2020-05-09T05:03:53.000+0000,0,11.0,2,0
982631,661484,1249118,498922,349115,2019-10-30T17:07:12.013+0000,1,5.0,5,5
983595,690485,1248471,498927,349115,2020-01-28T14:48:20.090+0000,18,20.0,20,20


In [15]:
intervalFields


Out[44]: ['assignment_due_date',
 'assignment_final_submission_date',
 'assignment_start_date',
 'max_student_stop_datetime',
 'min_student_start_datetime',
 'scored_datetime',
 'student_start_datetime',
 'student_stop_datetime',
 'was_fully_scored_datetime',
 'was_in_progress_datetime',
 'was_submitted_datetime_actual']

In [16]:
spark.sql("SELECT * FROM clean_data LIMiT 20").printSchema()

root
-- ced_assignment_type_code: string (nullable = true)
-- is_affecting_grade: boolean (nullable = true)
-- number_of_learners: integer (nullable = true)
-- number_of_distinct_instance_items: integer (nullable = true)
-- assignment_max_attempts: integer (nullable = true)
-- assignment_final_submission_date: timestamp (nullable = true)
-- assignment_start_date: timestamp (nullable = true)
-- assignment_due_date: timestamp (nullable = true)
-- min_student_start_datetime: timestamp (nullable = true)
-- max_student_stop_datetime: timestamp (nullable = true)
-- assignment_attempt_number: integer (nullable = true)
-- was_fully_scored_datetime: timestamp (nullable = true)
-- was_submitted_datetime_actual: timestamp (nullable = true)
-- was_in_progress_datetime: timestamp (nullable = true)
-- is_force_scored: boolean (nullable = true)
-- is_manual_scoring_required: boolean (nullable = true)
-- student_start_datetime: timestamp (nullable = true)
-- student_stop_datetime: timestamp (nullable = true)
-- item_is_offline_scored: boolean (nullable = true)
-- learner_attempt_status: string (nullable = true)
-- points_possible_unweighted: double (nullable = true)
-- final_score_unweighted: integer (nullable = true)
-- scored_datetime: timestamp (nullable = true)
-- item_type_code_name: string (nullable = true)
-- scoring_type_code: string (nullable = true)
-- response_correctness: string (nullable = true)
-- assigned_item_status: string (nullable = true)
-- org_id: integer (nullable = true)
-- section_id: integer (nullable = true)
-- assessment_id: integer (nullable = true)
-- assessment_instance_id: integer (nullable = true)
-- learner_assignment_attempt_id: integer (nullable = true)
-- assessment_instance_attempt_id: integer (nullable = true)
-- learner_id: integer (nullable = true)
-- learner_assigned_item_attempt_id: integer (nullable = true)
-- assessment_item_response_id: integer (nullable = true)